In [1]:
from scipy.stats import entropy
from collections import Counter
import numpy as np
import os
from tqdm.notebook import tqdm
import pandas as pd
import pickle

In [2]:
def get_entropy(byte_array):
    counts = [0] * 256
    for byte in byte_array:
        counts[byte] += 1
    counts = np.array(counts)
    ent = entropy(counts, base=2)
    
    return (ent, counts )

print(get_entropy(b"y"))

(0.0, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))


In [3]:
def get_substrings(s, min_length, max_length):
    """Generate substrings of a given string within a specified length range."""
    length = len(s)
    return [s[i:j] for i in range(length) for j in range(i + min_length, min(i + max_length + 1, length + 1))]

def most_common_substrings(strings, min_substring_length=4, max_substring_length=10, min_occurrences=3):
    """Return the most common substrings sorted by frequency and alphabetically, 
    excluding those that are substrings of other returned values."""
    substrings = []
    for string in tqdm(strings, desc="Extending Strings"):
        substrings.extend(get_substrings(string, min_substring_length, max_substring_length))

    # Count the frequency of each substring
    substring_count = Counter(substrings)
    print(len(substrings))
    
    # Filter substrings based on minimum occurrence threshold
    print("Enforcing minimum occurrence")
    filtered_substrings = {substring: count for substring, count in substring_count.items() if count >= min_occurrences}

    # Sort by frequency (descending) and alphabetically
    print("Sorting...")
    sorted_substrings = sorted(filtered_substrings, key=lambda x: (-filtered_substrings[x], x))
    sorted_substrings = sorted_substrings[:10000]
    # Remove substrings that are part of longer substrings in the list
    final_substrings = []
    for substring in tqdm(sorted_substrings, desc="Filtering Strings"):
        if not any(substring in other and substring != other for other in sorted_substrings):
            final_substrings.append(substring)

    return final_substrings


# Example usage
strings = ["testing", "this is a test", "foobar", "fazbar", "what is foobar", "can afoobart the testers"]
print(most_common_substrings(strings))

Extending Strings:   0%|          | 0/6 [00:00<?, ?it/s]

260
Enforcing minimum occurrence
Sorting...


Filtering Strings:   0%|          | 0/7 [00:00<?, ?it/s]

['foobar', 'test']


In [4]:

def create_unique_ngrams_set(strings, n):
    def generate_ngrams(s):
        return [s[i:i + n] for i in range(len(s) - n + 1)]

    all_ngrams = set()
    for s in tqdm(strings, desc="Creating unique n-grams"):
        ngrams = generate_ngrams(s)
        all_ngrams.update(ngrams)
    return list(all_ngrams)  # Convert to list to maintain order

def count_ngrams_in_string(s, ngrams, n):
    ngram_counts = Counter(s[i:i + n] for i in range(len(s) - n + 1))
    return [ngram_counts[ngram] for ngram in ngrams]

def get_ngram_by_index(ngrams, index):
    if 0 <= index < len(ngrams):
        return ngrams[index]
    else:
        return "Index out of range"

# Example usage
n = 4
strings = ["hello world", "world of python", "python programming"]
unique_ngrams = create_unique_ngrams_set(strings, n)
print(unique_ngrams)

test_string = "hello "
ngram_counts = count_ngrams_in_string(test_string, unique_ngrams, n)
print(ngram_counts)

# Get the n-gram at a specific index
for i, x in enumerate(ngram_counts):
    if x > 0: 
        ngram_at_index = get_ngram_by_index(unique_ngrams, i)
        print(f"N-gram at index {i}: {ngram_at_index}")

Creating unique n-grams:   0%|          | 0/3 [00:00<?, ?it/s]

['on p', 'ramm', 'rogr', 'ming', 'o wo', 'ogra', ' of ', 'hell', 'n pr', 'llo ', 'f py', 'ytho', 'lo w', 'thon', ' wor', 'prog', 'rld ', 'worl', 'orld', 'd of', 'gram', 'of p', 'pyth', ' pyt', ' pro', 'ammi', 'ld o', 'mmin', 'ello', 'hon ']
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
N-gram at index 7: hell
N-gram at index 9: llo 
N-gram at index 28: ello


# Process corpus from fuzzing results

In [5]:
data_nominal = "./data/fuzzingprotection/libxml2_nominal500k/"
data_crash = "./data/fuzzingprotection/libxml2_crash500k/"

def read_files_to_byte_array(directory):
    file_contents = []
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

    # Using tqdm to show progress
    for filename in tqdm(files, desc="Reading files"):
        file_path = os.path.join(directory, filename)

        with open(file_path, 'rb') as file:
            content = file.read()
            file_contents.append(content)

    return list(set(file_contents))



In [6]:
data = read_files_to_byte_array(data_nominal)
labels = [0]*len(data)
data += read_files_to_byte_array(data_crash)
labels += [1]* (len(data) - len(labels))

Reading files:   0%|          | 0/541746 [00:00<?, ?it/s]

Reading files:   0%|          | 0/521279 [00:00<?, ?it/s]

In [7]:
# Metadata about the dataset

max_len = max([len(d) for d in data])
unique_bytes = set()
for d in tqdm(data, desc="Analyzing Samples"):
    for b in d: unique_bytes.add(b)
    



Analyzing Samples:   0%|          | 0/1042907 [00:00<?, ?it/s]

In [8]:
common_substrings = most_common_substrings(data,min_occurrences=100)[:100]


Extending Strings:   0%|          | 0/1042907 [00:00<?, ?it/s]

587475668
Enforcing minimum occurrence
Sorting...


Filtering Strings:   0%|          | 0/10000 [00:00<?, ?it/s]

In [9]:
#n = 3
#unique_ngrams = create_unique_ngrams_set(data, n)

In [10]:
#len(unique_ngrams)

In [11]:
chr(97)

'a'

In [12]:
row_names = ["entropy"]
for i in range(256):
    row_names.append(chr(i))
for x in common_substrings:
    row_names.append(x)
row_names.append("sample")
row_names.append("crash")


In [13]:
max_byte_counts = [0]*256
max_ent = 0

for d in tqdm(data, "Finding maximums"):
    ent, bs = get_entropy(d) 
    if ent > max_ent: max_ent = ent
    for i, c in enumerate(bs):
        if c > max_byte_counts[i]: max_byte_counts[i] = c
max_byte_counts = np.array(max_byte_counts, dtype="float64")

Finding maximums:   0%|          | 0/1042907 [00:00<?, ?it/s]

In [14]:
# Process each sample

features = []
for d,l in tqdm(zip(data,labels), total=len(data), desc="Processing samples"):
    substrings = [0]*100
    ent, bs = get_entropy(d)
    bs = bs.astype('float64') 
    ent /= max_ent
    bs /= max_byte_counts
    for i, s in enumerate(common_substrings):
        if s in d: substrings[i] = 1
    
    row = [ent]
    row += list(bs)
    row += list(substrings)
    row += [d]
    row += [l] # Training labels must be last
    features.append( row ) 

Processing samples:   0%|          | 0/1042907 [00:00<?, ?it/s]

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


In [15]:
len(row_names)

359

In [16]:
len(features[0])

359

In [18]:
df = pd.DataFrame(features, columns=row_names)

In [19]:
df.to_pickle("data/libxml2_train.pkl")

In [20]:
df2 = pd.read_pickle("data/libxml2_train.pkl")

In [21]:
df2

,entropy, ,,,,,,,,,...,b'!ENTITYd<!',b'\xff\xfe\xfe\xfe@\x80\x00\xd9<?',b'ENTITYd<!A',b'ITYd<!ATTL',"b'rsion=""2.5'",b'NTITYd<!AT',b'TITYd<!ATT',"b'rsion=""1.0'",sample,crash
0,0.645256,0.021583,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,b'<!DOCTYPE%;%[G\xff;\x00<\xcd<\x8fDOCTYPE\xff...,0
1,0.687793,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,b'<!D<!ATTL<!DOCTYPETYdOCTYPEU\\[<!ENTIU\\[<!E...,0
2,0.801382,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,1,0,1,1,0,1,1,0,b'<Yxml xml:spaceml:dio7=\'g\n\n\n\n\n\n\n\n\n...,0
3,0.782414,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.007246,0.0,0.0,...,0,0,0,0,0,0,0,0,"b'<oldpEndMa>E<![CDA<l x\xedlns=""/>)3) xmlns=""...",0
4,0.779685,0.035971,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,1,0,0,0,0,0,b'<!D\x00\x00\x00\x00PEUrsCTY\xffEv\\[U\xff\xf...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042902,0.642829,0.021583,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,1,0,0,0,0,0,0,b'\xff\xfe\xfe\xfe@\x80\x00\xd9<?xml encofing=...,1
1042903,0.680316,0.122302,0.0,0.007353,0.0,0.007246,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,b'\xfe\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00...,1
1042904,0.762516,0.007194,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,0,0,1,0,0,0,b'\xff\xfe\xfe\xfeS\xfe\xfe\xfe@\xf5\xd9\xd9<?...,1
1042905,0.569356,0.050360,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,0,0,1,0,0,0,"b'<?xml encoding=""UK\x00xl\xd1\xfe\xff\xff\xfe...",1


In [22]:
with open("data/libxml2_metadata.pkl", "wb") as f:
    pickle.dump({"max_ent": max_ent, "max_byte_counts": max_byte_counts }, f)